In [1]:
!pip install -U bitsandbytes
!pip install accelerate
!pip install PyPDF2
!pip install pinecone-client
!pip install fpdf
!pip install huggingface_hub
!pip install transformers
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=c43d20f009926f3b17700b3567923bffe6b4ed5e605fb26ac9ba1af35bb42baf
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf


In [2]:
import os
from PyPDF2 import PdfReader
import math
from transformers import (
    MBartForConditionalGeneration,
    MBart50TokenizerFast,
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline
)
from getpass import getpass
import torch
from torch.cuda.amp import autocast

In [3]:
# Function to read the PDF content
def read_pdf(file_path):
    """
    Reads text from a PDF file.

    Args:
        file_path (str): Path to the PDF file.

    Returns:
        str: Combined text of all pages.
    """
    reader = PdfReader(file_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"
    return text.strip()

In [4]:
# Function to split content into chunks
def create_chunks(text, chunk_size, overlap):
    """
    Splits text into chunks with overlapping context.

    Args:
        text (str): The input text.
        chunk_size (int): The maximum size of each chunk (characters).
        overlap (int): The number of overlapping characters between chunks.

    Returns:
        list: List of text chunks.
    """
    words = text.split()
    chunks = []
    current_chunk = []
    current_length = 0

    for word in words:
        word_length = len(word) + 1
        if current_length + word_length > chunk_size:
            chunks.append(" ".join(current_chunk))
            overlap_words = (
                current_chunk[-math.ceil(overlap / len(current_chunk)) :]
                if current_chunk
                else []
            )
            current_chunk = overlap_words
            current_length = sum(len(w) + 1 for w in current_chunk)

        current_chunk.append(word)
        current_length += word_length

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

In [5]:
file_path = "/content/CASE OF ANAGNOSTAKIS v. GREECE.pdf"
document_text = read_pdf(file_path)
chunks = create_chunks(document_text, chunk_size=3000, overlap=500)

In [6]:
len(chunks)

21

In [7]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass("Enter your Hugging Face API token: ")

Enter your Hugging Face API token: ··········


In [8]:
model_name = "facebook/mbart-large-50-many-to-one-mmt"
# model = MBartForConditionalGeneration.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")
model = MBartForConditionalGeneration.from_pretrained(model_name)
# tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/268 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): MBartScaledWordEmbedding(250054, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): MBartScaledWordEmbedding(250054, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
    

In [10]:
# Function to translate chunks into English
def translate_chunks(chunks, batch_size):
    """
    Translates a list of text chunks into English using MBart50.

    Args:
        chunks (list): List of text chunks in the source language.
        batch_size (int): Number of chunks to process in a single batch.

    Returns:
        list: Translated text chunks in English.
    """
    translations = []
    for i in range(0, len(chunks), batch_size):
        batch = chunks[i:i + batch_size]
        encoded_input = tokenizer(batch, return_tensors="pt", padding=True, truncation=True).to(model.device)

        with autocast():
            generated_tokens = model.generate(**encoded_input, max_length=128, num_beams=1)

        translations.extend(tokenizer.batch_decode(generated_tokens, skip_special_tokens=True))
    return translations

In [11]:
translated_chunks = translate_chunks(chunks, batch_size=8)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
<ipython-input-10-6a8f68642c05>:18: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


In [12]:
for i, translation in enumerate(translated_chunks, 1):
    print(f"Chunk {i} Translation:\n{translation}\n")

Chunk 1 Translation:
THIRD SECTION CASE OF ANAGNOSTAKIS v. GREECE (Application no. 26504/20) JUDGMENT Art 8 • Positive obligations • Family life • Non-enforcement of judicial decisions granting applicant contact rights with his child not attributed to a lack of diligence on the part of the relevant authorities • Authorities’ efforts in vain owing to tense relationship between parents and their behaviour, including applicant’s unaccommodating conduct STRASBOURG 10 October 2023 This judgment will become final in the circumstances set out in Article 44

Chunk 2 Translation:
20 October 2014 the applicant voluntarily recognised the child as his own through a declaration before a notary with E.K.’s consent. By this act, the parents agreed that their son would have the applicant’s last name and that they would exercise joint parental responsibility. 7. The applicants lived together until the beginning of October 2014 when, according to the applicant, E.K. forced him to leave. The child contin

In [13]:
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone, ServerlessSpec

In [14]:
pc = Pinecone(api_key=getpass("Enter your Pinecone API token: "))

Enter your Pinecone API token: ··········


In [15]:
index_name = "legal-llm"
if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

index = pc.Index(index_name)

In [16]:
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

embeddings = embedding_model.encode(translated_chunks)

for i, chunk in enumerate(translated_chunks):
    index.upsert(vectors=[(f"doc_{i}", embeddings[i], {"text": chunk})])

print("Translated chunks embedded and stored in Pinecone.")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Translated chunks embedded and stored in Pinecone.


In [17]:
from huggingface_hub import login

login(token=getpass("Enter your Hugging Face access token: "))

Enter your Hugging Face access token: ··········


In [18]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")

# Configure the model for 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

# Load the model with the BitsAndBytesConfig
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-chat-hf",
    quantization_config=bnb_config,
    device_map="auto",
)

# Create a pipeline for text generation
llm = pipeline("text-generation", model=model, tokenizer=tokenizer)

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Device set to use cuda:0


In [19]:
# Prompt template
PROMPT_TEMPLATE = """You are a highly skilled legal expert with deep knowledge of law and legal cases.
You provide accurate, well-reasoned, and reliable answers based strictly on the given legal documents.

Context:
{context}

Question:
{question}

Instructions:
- Answer the question strictly based on the given context.
- If the context does not contain enough information, state that explicitly.
- Provide detailed legal reasoning where necessary.
- Keep the response clear and professional.
"""

In [20]:
# Function to perform similarity search
def search_similar_documents(query, index, embedding_model, top_k=5):
    """
    Searches for the most relevant documents in the Pinecone index based on query similarity.

    Args:
        query (str): The input legal question.
        index (pinecone.Index): The Pinecone index instance.
        embedding_model (SentenceTransformer): The embedding model.
        top_k (int): Number of top similar documents to retrieve.

    Returns:
        list: Retrieved similar documents.
    """
    query_embedding = embedding_model.encode(query).tolist()
    results = index.query(vector=query_embedding, top_k=top_k, include_metadata=True)

    return [match['metadata']['text'] for match in results['matches']]

In [21]:
# Function to generate response
import re

def generate_legal_response(question, index, embedding_model, llm, top_k=5):
    """
    Generates a legal response by retrieving relevant documents and using the LLM.

    Args:
        question (str): The legal question asked by the user.
        index (pinecone.Index): The Pinecone index instance.
        embedding_model (SentenceTransformer): The embedding model.
        llm (Pipeline): The LLM text generation pipeline.
        top_k (int): Number of relevant document chunks to retrieve.

    Returns:
        str: The legal response generated by the LLM.
    """
    # Retrieve relevant context from the vector database
    relevant_docs = search_similar_documents(question, index, embedding_model, top_k)

    if not relevant_docs:
        return "I couldn't find relevant legal information in the provided documents."

    context = "\n\n".join(relevant_docs)

    # Prompt template
    prompt = PROMPT_TEMPLATE.format(context=context, question=question)

    # Generate response
    response = llm(prompt, max_new_tokens=256, do_sample=True, temperature=0.7)[0]['generated_text']

    # Remove any parts of the prompt accidentally included in the output
    cleaned_response = response.replace(prompt, "").strip()

    # Remove "Legal Answer:", "Context:", and "Instructions:" titles and content if they appear in the original response
    cleaned_response = re.sub(r"(?i)(Legal Answer:|Context:|Instructions:)", "", cleaned_response).strip()

    return cleaned_response

In [22]:
question = "What were the key legal arguments in the case ANAGNOSTAKIS v. GREECE?"

response = generate_legal_response(question, index, embedding_model, llm, top_k=5)

print(response)

Based on the given context, the key legal arguments in the case ANAGNOSTAKIS v. GREECE can be summarized as follows:

1. Positive obligations: The court found that Greece had a positive obligation to protect the applicant's family life, as guaranteed by Article 8 of the European Convention on Human Rights. The authorities' failure to enforce the applicant's contact rights with his child was seen as a violation of this positive obligation.
2. Non-enforcement of judicial decisions: The court noted that the authorities' failure to enforce the judicial decisions granting the applicant contact rights with his child was not attributed to a lack of diligence on the part of the relevant authorities. Instead, the court found that the authorities' efforts were in vain due to the tense relationship between the applicant and his son, including the applicant's unaccommodating conduct.
3. Family life: The court emphasized the importance of the applicant's family life and the need to protect it from 

In [23]:
# question = "What was the main issue in the case of C.P. and M.N. v. France?"

# response = generate_legal_response(question, index, embedding_model, llm, top_k=5)

# print(response)

In [24]:
# question = "Why did the French courts declare the applicant’s paternity challenge inadmissible?"

# response = generate_legal_response(question, index, embedding_model, llm, top_k=5)

# print(response)

In [25]:
# question = "What was the European Court of Human Rights’ (ECtHR) decision in this case?"

# response = generate_legal_response(question, index, embedding_model, llm, top_k=5)

# print(response)